# Tutorial for sampling using Qiskit
In this tutorial, we will look at tutorials of implementations using **Qiskit**. Participants are recommended to read the `sampling.ipynb` beforehand.

The sampling function which we provided supports input of **Qiskit circuits** and **operators**. Let's actually implement it.

## ChallengeSampling

In [11]:
import sys

sys.path.append("../")
from utils.challenge_2023 import ChallengeSampling

# define challenge_sampling with or without noise
challenge_sampling = ChallengeSampling(noise=True)

### Prepare a Qiskit circuit

In [12]:
from qiskit import QuantumCircuit as QiskitQuantumCircuit
from math import pi

qiskit_circuit = QiskitQuantumCircuit(4)
qiskit_circuit.x(0)
qiskit_circuit.h(1)
qiskit_circuit.y(2)
qiskit_circuit.cx(1, 2)
qiskit_circuit.rx(pi/4, 3)

# Sampler
 By using the instance challenge_sampling, you can construct the sampler. This sampler can take Qiksit circuits as input.

In [22]:
# choose hardware type
hardware_type = "sc"

# create sampler
sampler = challenge_sampling.create_sampler(hardware_type)
# possible to choose Qiksit circuits as an input
sampling_result = sampler(qiskit_circuit, n_shots=1000)
print(f"counts: {sampling_result}")

counts: Counter({5: 443, 3: 325, 13: 102, 11: 81, 7: 13, 4: 11, 1: 9, 2: 7, 9: 6, 15: 2, 10: 1})


A sampler receives two arguments: a circuit to be sampled and a number of repeated samplings (shots). The returned value is a mapping (dict) with the following keys and values:

- **Keys** Bitstrings of measurement outcomes, encoded as int. Each measured bit of each qubit are ordered from the least significant bit to the most significant bit. For example, when qubit 0 and qubit 2 are measured to be in  while the others are in , the bitstring is 0b0101.

- **Values** Counts of times for which each bitstring is measured. A sum of all counts is equal to the specified shots.

## Concurrent sampler
The concurrent sampler can take Qiksit circuits as input as same as the usual sampler.

In [14]:
# another circuit with 4 qubits
qiskit_circuit2 = QiskitQuantumCircuit(4)
qiskit_circuit2.t(0)
qiskit_circuit2.h(1)
qiskit_circuit2.ry(3*pi/4, 2)
qiskit_circuit2.cx(1, 2)
qiskit_circuit2.rz(pi, 3)

# create concurrent sampler
concurrent_sampler = challenge_sampling.create_concurrent_sampler(hardware_type)
concurrent_counts = concurrent_sampler([(qiskit_circuit, 1000), (qiskit_circuit2, 1000)])

print(f"concurrent_counts: {concurrent_counts}")

concurrent_counts: [Counter({5: 420, 3: 416, 11: 62, 13: 60, 1: 21, 7: 7, 2: 7, 9: 2, 4: 2, 15: 2, 10: 1}), Counter({2: 451, 4: 418, 0: 68, 6: 57, 12: 4, 1: 1, 8: 1})]


# Sampling Estimator
In order to estimate an expectation value of operators involves operators and states, you need to use a sampling_estimator. You can construct the concurrent sampler as same as the usual sampler with some additional inputs. 

The sampling_estimator can take Qiksit circuits and Qiskit operators as inputs. First we define a qiskit operator to be estimated.

In [15]:
# define a qiskit operator to be estimated
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.opflow import PauliSumOp


qiskit_paulisumop = PauliSumOp(primitive=SparsePauliOp(Pauli("ZIII")), coeff=0.25)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IZZI")), coeff=2.0)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IXXI")), coeff=0.5 + 0.25j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IZIY")), coeff=1.0j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IIZY")), coeff=1.5 + 0.5j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IXIY")), coeff=2.0j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IIII")), coeff=3.0)
print(qiskit_paulisumop)


qiskit_paulisumop2 = PauliSumOp(primitive=SparsePauliOp(Pauli("ZIII")), coeff=0.25 + 1.22j)
qiskit_paulisumop2 += PauliSumOp(primitive=SparsePauliOp(Pauli("IZZI")), coeff=1.4)
qiskit_paulisumop2 += PauliSumOp(primitive=SparsePauliOp(Pauli("IIZZ")), coeff=0.2)
print(qiskit_paulisumop2)

0.25 * ZIII
+ 2.0 * IZZI
+ (0.5+0.25j) * IXXI
+ 1j * IZIY
+ (1.5+0.5j) * IIZY
+ 2j * IXIY
+ 3.0 * IIII
(0.25+1.22j) * ZIII
+ 1.4 * IZZI
+ 0.2 * IIZZ


### Converting operator and circuits directly to quri_parts hamiltonian and circuits
You can convert qiskit operator and circuit directly using operator_from_qiskit_op and circuit_from_qiskit, respectively. Examples are shown below.

In [16]:
from quri_parts.qiskit.circuit import circuit_from_qiskit
from quri_parts.qiskit.operator import operator_from_qiskit_op

##: prepare qiskit operator
qp_op = operator_from_qiskit_op(pauli_operator=qiskit_paulisumop)
qp_op2 = operator_from_qiskit_op(pauli_operator=qiskit_paulisumop2)

##: prepare circuit
qp_circuit = circuit_from_qiskit(qiskit_circuit=qiskit_circuit)
qp_circuit2 = circuit_from_qiskit(qiskit_circuit=qiskit_circuit2)

## Pauli grouping
The operator is represented as a sum of Pauli operators. One of the ways to estimate expectation value of such an operator is to estimate expectation value of each Pauli term and then sum up them.

When estimating the Pauli terms, it is possible to measure multiple Pauli terms at once if they are commutable. The first step is thus to group the Pauli terms into several sets of commutable Pauli terms. This Pauli grouping is an important research subject in context of operator estimation.

One of the simplest Pauli grouping is bitwise grouping, where the groups are determined based on bitwise commutability of the Pauli terms. We can test the grouping as follows:

In [17]:
from quri_parts.core.operator.grouping import bitwise_pauli_grouping
pauli_sets = bitwise_pauli_grouping(qp_op)
print(f"Number of groups: {len(pauli_sets)}")
for i, pauli_set in enumerate(pauli_sets):
    labels = ", ".join([str(pauli) for pauli in pauli_set])
    print(f"Group {i} contains: {labels}")

Number of groups: 5
Group 0 contains: X1 Y3
Group 1 contains: I
Group 2 contains: X1 X2
Group 3 contains: Z0, Z1 Z2
Group 4 contains: Z2 Y3, Z1 Y3


## Shot allocator
Another input necessary for estimation is PauliSamplingShotsAllocator: it specifies how total sampling shots should be allocated to measurement of each Pauli sets. There are several allocators available:

In [18]:
from quri_parts.core.sampling.shots_allocator import (
    create_equipartition_shots_allocator,
    create_proportional_shots_allocator,
    create_weighted_random_shots_allocator,
)
# Allocates shots equally among the Pauli sets
allocator = create_equipartition_shots_allocator()
# Allocates shots proportional to Pauli coefficients in the operator
allocator = create_proportional_shots_allocator()
# Allocates shots using random weights
allocator = create_weighted_random_shots_allocator(seed=777)

### Estimate expectation value by sampling
With these inputs and qiskit operators and qiskit circuits, sampling estimation can be performed as follows:

In [19]:
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement

shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement
n_shots = 10000

# returns estimated value using above inputs
estimated_value = challenge_sampling.sampling_estimator(
    operator=qiskit_paulisumop,
    state_or_circuit=qiskit_circuit,
    n_shots=10000,
    measurement_factory=measurement_factory,
    shots_allocator=shots_allocator,
    hardware_type="sc",
)

# returns real part of estimated value
print(f"estimated_value :{estimated_value.value.real} ")

estimated_value :1.2953999999999999 


You can also construct the concurrent sampling_estimator as follows:

In [20]:
# create concurrent sampling estimator
concurrent_estimated_value = challenge_sampling.concurrent_sampling_estimator(
    operators=[qiskit_paulisumop, qiskit_paulisumop2],
    states=[qiskit_circuit, qiskit_circuit2],
    total_shots=10000,
    measurement_factory=measurement_factory,
    shots_allocator=shots_allocator,
    hardware_type="sc",
)


print(f"concurrent_estimated_value :{concurrent_estimated_value[0].value.real, concurrent_estimated_value[1].value.real} ")

concurrent_estimated_value :(1.0274, -0.78604) 
